In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
# Define custom dataset class
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = self.data.iloc[idx, :-1].values.astype(float)
        label = self.data.iloc[idx, -1]
        return {'features': torch.tensor(features), 'label': torch.tensor(label)}

csv_file = 'build/HandMCout.csv'  # Replace 'data.csv' with the path to your CSV file
dataset = CustomDataset(csv_file)

print(dataset[0])
# Split dataset into train and test sets
#train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)
#print(test_set)


{'features': tensor([ 8.,  4., 13.,  4., 10.,  1.,  5.,  4.,  6.,  2.], dtype=torch.float64), 'label': tensor(0.4510, dtype=torch.float64)}


In [28]:
# Define the neural network
class FunctionApproximator(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FunctionApproximator, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Load CSV fil
# Define model, loss function, and optimizer
input_dim = len(train_set[0]['features'])
output_dim = 1  # Assuming a single output
hidden_dim = 50 # Adjust according to your problem
model = FunctionApproximator(input_dim, hidden_dim, output_dim)
criterion = nn.MSELoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

# Train the model
num_epochs = 50
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs):
    for batch in train_loader:
        features, labels = batch['features'], batch['label']
        optimizer.zero_grad()
        outputs = model(features.float())
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
test_loss = 0
with torch.no_grad():
    for batch in test_loader:
        features, labels = batch['features'], batch['label']
        outputs = model(features.float())
        test_loss += criterion(outputs.squeeze(), labels.float()).item()

test_loss /= len(test_loader)
print(f'Test Loss: {test_loss:.4f}')

Epoch [1/50], Loss: 0.0463
Epoch [2/50], Loss: 0.0207
Epoch [3/50], Loss: 0.0243
Epoch [4/50], Loss: 0.0190
Epoch [5/50], Loss: 0.0365
Epoch [6/50], Loss: 0.0339
Epoch [7/50], Loss: 0.0339
Epoch [8/50], Loss: 0.0126
Epoch [9/50], Loss: 0.0300
Epoch [10/50], Loss: 0.0130
Epoch [11/50], Loss: 0.0211
Epoch [12/50], Loss: 0.0242
Epoch [13/50], Loss: 0.0119
Epoch [14/50], Loss: 0.0261
Epoch [15/50], Loss: 0.0507
Epoch [16/50], Loss: 0.0570
Epoch [17/50], Loss: 0.0142
Epoch [18/50], Loss: 0.0555
Epoch [19/50], Loss: 0.0272
Epoch [20/50], Loss: 0.0181
Epoch [21/50], Loss: 0.0349
Epoch [22/50], Loss: 0.0236
Epoch [23/50], Loss: 0.0138
Epoch [24/50], Loss: 0.0267
Epoch [25/50], Loss: 0.0291
Epoch [26/50], Loss: 0.0152
Epoch [27/50], Loss: 0.0212
Epoch [28/50], Loss: 0.0147
Epoch [29/50], Loss: 0.0328
Epoch [30/50], Loss: 0.0249
Epoch [31/50], Loss: 0.0212
Epoch [32/50], Loss: 0.0161
Epoch [33/50], Loss: 0.0205
Epoch [34/50], Loss: 0.0055
Epoch [35/50], Loss: 0.0266
Epoch [36/50], Loss: 0.0450
E

In [27]:
manualinput = torch.tensor([[5,2,2,1,12,3,7,3,11,4]]).float()
greathand = torch.tensor([[14,1,14,2,13,4,12,2,11,5]]).float()

model.eval()
with torch.no_grad():
    output = model(manualinput)
print(output)
with torch.no_grad():
    output = model(greathand)
print(output)

tensor([[0.2106]])
tensor([[1.0542]])
